In [16]:
>>> from random import choice
>>> x = choice(['Hello, world!', [1, 2, 'e', 'e', 4]])
x


'Hello, world!'

In [17]:
>>> x.count('e')


1

上面的x可以是字符串，也可是列表，但count可以自动
匹配，这就是多态处理。

如你所见，这个函数还使用了repr。repr是多态的集大成者之一，可用于任何对象，

In [18]:
def length_message(x):
     print("The length of", repr(x), "is", len(x))


In [19]:
>>> length_message('Fnord')


The length of 'Fnord' is 5


In [20]:
>>> length_message([1, 2, 3])


The length of [1, 2, 3] is 3


In [22]:
>>> length_message(((1, 2, 3),12))


The length of ((1, 2, 3), 12) is 2


In [24]:
class Person:
            def set_name(self, name):
                self.name = name


            def get_name(self):
                return self.name


            def greet(self):
                print("Hello, world! I'm {}.".format(self.name))


In [25]:
>>> foo = Person()
>>> bar = Person()
>>> foo.set_name('Luke Skywalker')
>>> bar.set_name('Anakin Skywalker')
>>> foo.greet()
        

Hello, world! I'm Luke Skywalker.


In [26]:
bar.greet()

Hello, world! I'm Anakin Skywalker.


In [27]:
foo.name


'Luke Skywalker'

In [28]:
>>> bar.name = 'Yoda'
>>> bar.greet()


Hello, world! I'm Yoda.


要让方法或属性成为私有的（不能从外部访问），只需让其名称以两个下划线打头即可。

In [29]:
class Secretive:
        def __inaccessible(self):
            print("Bet you can't see me ...")


        def accessible(self):
            print("The secret message is:")
            self.__inaccessible()


In [30]:
>>> s = Secretive()
>>> s.__inaccessible()


AttributeError: 'Secretive' object has no attribute '__inaccessible'

In [31]:
>>> s.accessible()


The secret message is:
Bet you can't see me ...


虽然以两个下划线打头有点怪异，但这样的方法类似于其他语言中的标准私有方法。然而，幕后的处理手法并不标准：在类定义中，对所有以两个下划线打头的名称都进行转换，即在开头加上一个下划线和类名。

In [32]:
>>> Secretive._Secretive__inaccessible


<function __main__.Secretive.__inaccessible(self)>

In [33]:
>>> s._Secretive__inaccessible()


Bet you can't see me ...


In [34]:
class MemberCounter:
            members = 0
            def init(self):
              MemberCounter.members += 1


In [35]:
>>> m1 = MemberCounter()
>>> m1.init()
>>> MemberCounter.members


1

In [36]:
>>> m2 = MemberCounter()
>>> m2.init()
>>> MemberCounter.members


2

In [2]:
class Filter:
        def init(self):
            self.blocked = []
        def filter(self, sequence):
            return [x for x in sequence if x not in self.blocked]


class SPAMFilter(Filter): # SPAMFilter是Filter的子类
        def init(self): # 重写超类Filter的方法init
            self.blocked = ['SPAM']


In [38]:
>>> f = Filter()
>>> f.init()
>>> f.filter([1, 2, 3])


[1, 2, 3]

In [39]:
>>> s = SPAMFilter()
>>> s.init()
>>> s.filter(['SPAM', 'SPAM', 'SPAM', 'SPAM', 'eggs', 'bacon', 'SPAM'])


['eggs', 'bacon']

请注意SPAMFilter类的定义中有两个要点。
❑ 以提供新定义的方式重写了Filter类中方法init的定义。
❑ 直接从Filter类继承了方法filter的定义，因此无需重新编写其定义。
第二点说明了继承很有用的原因：可以创建大量不同的过滤器类，它们都从Filter类派生而来，并且都使用已编写好的方法filter。这就是懒惰的好处。

In [5]:
>>> issubclass(Filter, SPAMFilter)


False

issubclass(SPAMFilter,Filter要确定一个类是否是另一个类的子类，可使用内置方法issubclass。)

如果你有一个类，并想知道它的基类，可访问其特殊属性__bases__。

In [7]:
>>> SPAMFilter.__bases__


(__main__.Filter,)

In [8]:
Filter.__base__

object

同样，要确定对象是否是特定类的实例，可使用isinstance。



In [9]:
>>> s = SPAMFilter()
>>> isinstance(s, SPAMFilter)


True

In [10]:
>>> isinstance(s, Filter)


True

如你所见，s是SPAMFilter类的（直接）实例，但它也是Filter类的间接实例，因为SPAMFilter是Filter的子类。换而言之，所有SPAMFilter对象都是Filter对象。从前一个示例可知，isinstance也可用于类型，如字符串类型（str）。

如果你要获悉对象属于哪个类，可使用属性__class__。

In [12]:
s.__class__

__main__.SPAMFilter

In [13]:
type(s)

__main__.SPAMFilter

7.2.8 多个超类
在前一节，你肯定注意到了一个有点奇怪的细节：复数形式的__bases__。前面说过，你可使用它来获悉类的基类，而基类可能有多个。为说明如何继承多个类，下面来创建几个类。

In [16]:
class Calculator:
    def calculate(self, expression):
           self.value = eval(expression)

class Talker:
    def talk(self):
        print('Hi, my value is', self.value)

class TalkingCalculator(Calculator, Talker):
        pass


In [17]:
>>> tc = TalkingCalculator()
>>> tc.calculate('1 + 2 * 3')
>>> tc.talk()


Hi, my value is 7


子类TalkingCalculator本身无所作为，其所有的行为都是从超类那里继承的。关键是通过从Calculator那里继承calculate，并从Talker那里继承talk，它成了会说话的计算器。

这被称为多重继承，是一个功能强大的工具。然而，除非万不得已，否则应避免使用多重继承，因为在有些情况下，它可能带来意外的“并发症”。

7.2.9 接口和内省
接口这一概念与多态相关。处理多态对象时，你只关心其接口（协议）——对外暴露的方法和属性。在Python中，不显式地指定对象必须包含哪些方法才能用作参数。例如，你不会像在Java中那样显式编写接口，而是假定对象能够完成你要求它完成的任务。如果不能完成，程序将失败。

In [18]:
>>> hasattr(tc, 'talk')


True

在上述代码中，你发现tc（本章前面介绍的TalkingCalculator类的实例）包含属性talk（指向一个方法），但没有属性fnord。如果你愿意，还可以检查属性talk是否是可调用的。

In [20]:
>>> callable(getattr(tc, 'talk', None))


True

注意setattr与getattr功能相反，可用于设置对象的属性：

In [22]:
>>> setattr(tc, 'name', 'Mr. Gumby')
>>> tc.name


'Mr. Gumby'

In [25]:
hasattr(tc,'name')

True

In [26]:
tc.__dict__

{'value': 7, 'name': 'Mr. Gumby'}

要查看对象中存储的所有值，可检查其__dict__属性。如果要确定对象是由什么组成的，应研究模块inspect。这个模块主要供高级用户创建对象浏览器（让用户能够以图形方式浏览Python对象的程序）以及其他需要这种功能的类似程序。有关对象和模块的详细信息，请参阅10.2节。

7.2.10 抽象基类
然而，有比手工检查各个方法更好的选择。在历史上的大部分时间内，Python几乎都只依赖于鸭子类型，即假设所有对象都能完成其工作，同时偶尔使用hasattr来检查所需的方法是否存在。很多其他语言（如Java和Go）都采用显式指定接口的理念，而有些第三方模块提供了这种理念的各种实现。最终，Python通过引入模块abc提供了官方解决方案。这个模块为所谓的抽象基类提供了支持。一般而言，抽象类是不能（至少是不应该）实例化的类，其职责是定义子类应实现的一组抽象方法。下面是一个简单的示例：

In [27]:
from abc import ABC, abstractmethod
class Talker(ABC):
            @abstractmethod
            def talk(self):
              pass


形如@this的东西被称为装饰器，其用法将在第9章详细介绍。这里的要点是你使用@abstractmethod来将方法标记为抽象的——在子类中必须实现的方法。

抽象类（即包含抽象方法的类）最重要的特征是不能实例化。

In [28]:
>>> Talker()


TypeError: Can't instantiate abstract class Talker with abstract method talk

In [34]:
class Knigget(Talker):
            pass


In [35]:
s=Knigget()

TypeError: Can't instantiate abstract class Knigget with abstract method talk

In [30]:
issubclass(Knigget,Talker)

True

In [38]:
class Knigget(Talker):
            def talk(self):
              print("Ni!")


In [39]:
s=Knigget()

由于没有重写方法talk，因此这个类也是抽象的，不能实例化。当上述改写后，又可实例化。

In [40]:
>>> isinstance(s, Talker)


True

In [41]:
s.talk()

Ni!


In [42]:
class Herring:
    def talk(self):
        print("Blub.")


In [43]:
>>> h = Herring()
>>> isinstance(h, Talker)


False

诚然，你可从Talker派生出Herring，这样就万事大吉了，但Herring可能是从他人的模块中导入的。在这种情况下，就无法采取这样的做法。为解决这个问题，你可将Herring注册为Talker（而不从Herring和Talker派生出子类），这样所有的Herring对象都将被视为Talker对象。

In [44]:
>>> Talker.register(Herring)


__main__.Herring

In [45]:
>>> isinstance(h, Talker)


True

In [46]:
>>> issubclass(Herring, Talker)


True

然而，这种做法存在一个缺点，就是直接从抽象类派生提供的保障没有了。

In [48]:
>>> class Clam:
          pass
          


In [49]:
Talker.register(Clam)


__main__.Clam

In [53]:
>>> issubclass(Clam, Talker)
>>> c = Clam()


In [54]:
>>> isinstance(c, Talker)


True

In [55]:
>>> c.talk()


AttributeError: 'Clam' object has no attribute 'talk'

换而言之，应将isinstance返回True视为一种意图表达。在这里，Clam有成为Talker的意图。本着鸭子类型的精神，我们相信它能承担Talker的职责，但可悲的是它失败了。